In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from helpers import imshow

In [2]:
""" Clear matches for which NN ratio is > than threshold """
def filter_distance(matches,ratio):
    dist = [m.distance for m in matches]
    thres_dist = (sum(dist) / len(dist)) * ratio

    # keep only the reasonable matches
    sel_matches = [m for m in matches if m.distance < thres_dist]
    #print '#selected matches:%d (out of %d)' % (len(sel_matches), len(matches))
    return sel_matches

""" keep only symmetric matches """
def filter_asymmetric(matches, matches2):
    sel_matches = []
    for match1 in matches:
        for match2 in matches2:
            if k_ftr[match1.queryIdx] == k_ftr[match2.trainIdx] and k_scene[match1.trainIdx] == k_scene[match2.queryIdx]:
                sel_matches.append(match1)
                break
    return sel_matches

# Todo: filter_ransac

def filter_matches(matches, matches2):
    matches = filter_distance(matches)
    matches2 = filter_distance(matches2)
    return filter_asymmetric(matches, matches2)

In [3]:
def orbTrack(trainImg,trainKP,trainDesc,QueryImgBGR,h,w):
    MIN_MATCH_COUNT=15

    detector=cv2.ORB_create()

    #FLANN_INDEX_KDITREE=0
    #flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
    #searchParam = dict(checks=50)
    #flann=cv2.FlannBasedMatcher(flannParam,searchParam)
    #flann=cv2.FlannBasedMatcher(flannParam,{})
    
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    #matches=flann.knnMatch(queryDesc,trainDesc,k=2)
    
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    #matches = bf.knnMatch(queryDesc,trainDesc)
    goodMatchA = bf.match(queryDesc,trainDesc)
    #goodMatchB = bf.match(trainDesc,queryDesc)
    goodMatch = filter_distance(goodMatchA,ratio=0.65)
    #goodMatchB = filter_distance(goodMatchB,ratio=0.65)
    #filter distance
    
    # Sort them in the order of their distance.
    # matches = sorted(matches, key = lambda x:x.distance)
    queryBorder = None
    qp = None
    #print(goodMatch[0]) 
    
    #goodMatch = filter_matches(goodMatchA,goodMatchB)
    print(len(goodMatch)) 
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        
        #h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
        return (1,queryBorder,qp)
        #print(queryBorder)
    else:
        #print("Not Enough match found")
        return (-1,queryBorder,qp)



In [4]:
def orbTrack2(trainImg,trainKP,trainDesc,QueryImgBGR,h,w):
    MIN_MATCH_COUNT=100

    detector=cv2.ORB_create()

    #FLANN_INDEX_KDITREE=0
    #flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
    #searchParam = dict(checks=50)
    #flann=cv2.FlannBasedMatcher(flannParam,searchParam)
    #flann=cv2.FlannBasedMatcher(flannParam,{})
    
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    #matches=flann.knnMatch(queryDesc,trainDesc,k=2)
    
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    #matches = bf.knnMatch(queryDesc,trainDesc)
    matchesF = bf.match(queryDesc,trainDesc)
    matchesB = bf.match(trainDesc,queryDesc)
    # Sort them in the order of their distance.
    #matches = sorted(matches, key = lambda x:x.distance)

    goodMatch=[]
    queryBorder = None
    qp = None
    for i in range(len(matchesF)):
        forward = matchesF[i]
        backward = matchesB[forward.trainIdx]
        if backward.trainIdx == forward.queryIdx:
            goodMatch.append(forward)
    
    print(len(goodMatch))
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        
        #h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
        return (1,queryBorder,qp)
        #print(queryBorder)
    else:
        #print("Not Enough match found")
        return (-1,queryBorder,qp)

In [5]:
def findOrientation(qp):
    tl = qp[0][0] #top left
    bl = qp[0][1] #botton left
    br = qp[0][2] #botton right
    tr = qp[0][3] #top right
    center = [(t1[0]+br[0])/2,(t1[1]+br[1])/2]
    x1 = bl[0]
    x2 = br[0]
    y1 = bl[1]
    y2 = br[1]
    if y1 <= y2:
        l = x2 - x1 
        h = y2 - y1
        theta = None
    else:
        l = x2 - x1
        h = y1 - y2
        theta = None
    
    #center is the center of the detected object
    #theta is the angle of it
    return center,theta
    

In [ ]:
cam = cv2.VideoCapture(0)
img1 = cv2.imread("demo.jpg")
h,w,d = img1.shape
detector = cv2.ORB_create()
kp1, des1 = detector.detectAndCompute(img1,None)
while True:
    ret, img2 = cam.read()
    checker,dst,qp = orbTrack(img1,kp1,des1,img2,h,w)
    #checker2,dst2,qp2 = orbTrack(img1,kp1,des1,img2,h,w)
    mask = img2.copy()
    if checker != -1:
        #center = findOrientation(qp)
        mask = cv2.polylines(img2,[np.int32(dst)],True,(0,255,0),5)
        #mask = cv2.polylines(mask,[np.int32(dst2)],True,(0,255,0),5)
        #print("YES")
    else:
        print("NO")
    keypressed = cv2.waitKey(5)
    if keypressed == 27:
        break
    cv2.imshow('mask',mask)
    
cam.release()
cv2.destroyAllWindows()

2
NO
2
NO
0
NO
2
NO
1
NO
1
NO
1
NO
1
NO
0
NO
0
NO
1
NO
1
NO
0
NO
6
NO
1
NO
2
NO
1
NO
3
NO
2
NO
0
NO
0
NO
3
NO
3
NO
3
NO
0
NO
0
NO
1
NO
1
NO
1
NO
2
NO
2
NO
2
NO
1
NO
1
NO
2
NO
1
NO
4
NO
1
NO
2
NO
2
NO
0
NO
2
NO
2
NO
1
NO
0
NO
3
NO
3
NO
4
NO
2
NO
3
NO
3
NO
3
NO
3
NO
1
NO
3
NO
2
NO
1
NO
2
NO
2
NO
2
NO
2
NO
2
NO
1
NO
0
NO
1
NO
2
NO
3
NO
1
NO
5
NO
25
35
29
24
21
18
NO
35
25
25
25
29
25
14
NO
17
NO
36
27
9
NO
18
NO
17
NO
16
NO
8
NO
8
NO
11
NO
20
NO
24
19
NO
26
35
38
48
48
45
51
48
52
42
28
16
NO
9
NO
6
NO
3
NO
5
NO
7
NO
11
NO
19
NO
26
29
30
24
22
28
26
25
30
14
NO
1
NO
1
NO
0
NO
2
NO
4
NO
4
NO
0
NO
13
NO
25
44
42
50
47
52
49
45
41
29
22
14
NO
7
NO
9
NO
10
NO
14
NO
10
NO
32
47
48
50
58
55
54
54
42
22
26
10
NO
18
NO
19
NO
37
53
47
41
48
48
46
50
50
48
10
NO
2
NO
6
NO
1
NO
22
51
57
59
58
55
52
48
50
53
54
33
26
50
24
4
NO
4
NO
3
NO
2
NO
3
NO
22
3
NO
2
NO
2
NO
2
NO
1
NO
2
NO
2
NO
1
NO
1
NO
5
NO
1
NO
3
NO
1
NO
1
NO
2
NO
4
NO
4
NO
3
NO
3
NO
2
NO
3
NO
1
NO
3
NO
4
NO
4
NO
1
NO
3
NO
2
NO
1
NO
1
NO
2
